In [1]:
import psycopg2
import pandas as pd
from psycopg2.extras import RealDictCursor

# Database connection parameters from docker-compose.yml
CONNECTION = 'host=postgres port=5432 dbname=mydb user=user password=password'

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor(cursor_factory=RealDictCursor)


In [2]:
cur.execute(f"SELECT * FROM stations")
result = cur.fetchall()
conn.commit()

In [11]:
result

[RealDictRow([('station_id', 'VS1721'),
              ('station_name', 'Stavanger (SW) - Risavika')]),
 RealDictRow([('station_id', 'VS1596'),
              ('station_name', 'Stavanger (S) - DSD')]),
 RealDictRow([('station_id', 'VS1595'),
              ('station_name', 'Stavanger (N) - Tjuvholmen')]),
 RealDictRow([('station_id', 'VS1722'),
              ('station_name', 'Stavanger (NW) - Mekjarvik')])]

In [3]:
query = """SELECT count(*) FROM stations"""
cur.execute(query)
result = cur.fetchone()
conn.commit()
result

RealDictRow([('count', 4)])

In [4]:

query = """SELECT last(wind_speed, timestamp) FROM wind_measurements"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
result

[RealDictRow([('last', 1.7)])]

In [5]:
query = """SELECT * FROM wind_measurements"""
query = """SELECT timestamp, wind_speed FROM wind_measurements WHERE timestamp BETWEEN '2025-03-09T07:10:33.387Z' AND '2025-03-09T23:40:33.387Z' ORDER BY 1"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
pd.DataFrame(result).sort_values('timestamp')

,timestamp,wind_speed
0,2025-03-09 21:20:13+00:00,1.0
1,2025-03-09 21:20:14+00:00,0.9
2,2025-03-09 21:20:15+00:00,0.7
3,2025-03-09 21:20:16+00:00,0.6
4,2025-03-09 21:20:17+00:00,0.5
...,...,...
8336,2025-03-09 21:59:32+00:00,2.2
8335,2025-03-09 21:59:32+00:00,2.1
8337,2025-03-09 21:59:33+00:00,1.9
8338,2025-03-09 21:59:34+00:00,1.7


In [9]:
query = """SELECT bucket as time, station_id, avg_speed as wind_speed, max_speed, min_speed FROM wind_measurements_15min"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
pd.DataFrame(result)

""


In [10]:
query = """SELECT
    time_bucket('1 hour', timestamp) AS bucket,
    station_id,
    avg(wind_speed) as avg_speed,
    max(wind_speed) as max_speed,
    min(wind_speed) as min_speed,
    -- Calculate the average direction using vector components
    degrees(point(avg(cos(radians(wind_direction))), avg(sin(radians(wind_direction)))) <-> point(0,0)) as avg_direction,
    count(*) as sample_count
FROM wind_measurements
GROUP BY bucket, station_id"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
pd.DataFrame(result)

,bucket,station_id,avg_speed,max_speed,min_speed,avg_direction,sample_count
0,2025-03-09 21:00:00+00:00,VS1596,1.119634,3.0,0.0,44.134860,2129
1,2025-03-09 22:00:00+00:00,VS1722,0.689032,2.3,0.0,30.909463,155
2,2025-03-09 21:00:00+00:00,VS1722,1.224262,3.0,0.0,41.674564,2065
3,2025-03-09 22:00:00+00:00,VS1721,0.765101,1.3,0.4,56.607078,149
4,2025-03-09 22:00:00+00:00,VS1595,2.175163,2.5,1.5,57.206869,153
5,2025-03-09 21:00:00+00:00,VS1721,0.477945,1.7,0.0,12.720279,2131
6,2025-03-09 22:00:00+00:00,VS1596,1.401987,2.5,0.3,54.815373,151
7,2025-03-09 21:00:00+00:00,VS1595,1.490597,2.5,0.4,47.086474,2127


# Test portwind connection

In [3]:
from requests import get 

In [11]:
station_id = "VS1721"

url = f"https://portwind.no/api/v1/dbdata-seconds.php?stationid={station_id}&dataset=wswd&seconds=60"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "Referer": "https://portwind.no/",
}
response = get(url, headers=header, timeout=10)
if response.status_code == 200:
    data = response.json()
data

ConnectTimeout: HTTPSConnectionPool(host='portwind.no', port=443): Max retries exceeded with url: /api/v1/dbdata-seconds.php?stationid=VS1721&dataset=wswd&seconds=60 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0xffff8ece89d0>, 'Connection to portwind.no timed out. (connect timeout=10)'))